# Day 16: Ticket Translation

[Brief](https://adventofcode.com/2020/day/16)

In [1]:
def parse_notes(notes):
    sections = notes.split("\n\n")
    fields = {}
    my_ticket = tuple()
    nearby_tickets = []
    
    for field in sections[0].split("\n"):
        f_name = field.split(": ")[0]
        f_ranges = [tuple([int(x) for x in r.split("-")]) for r in field.split(": ")[1].split(" or ")]
        fields[f_name] = f_ranges
    
    my_ticket = tuple([int(x) for x in sections[1].split("\n")[1].split(",")])
    nearby_tickets = [tuple([int(x) for x in ticket.split(",")]) for ticket in sections[2].split("\n")[1:]]
    
    return (fields, my_ticket, nearby_tickets)

In [14]:
def check_field(value, ranges):
    for r in ranges:
        if value >= r[0] and value <= r[1]:
            return True
    
    return False

In [13]:
def check_ticket(ticket, fields):
    invalid = []
    
    for value in ticket:
        # print("Testing {}...".format(value))
        is_valid_value = False
        for field in fields.keys():
            if check_field(value, fields[field]):
                # print(" - Valid   {}".format(field))
                is_valid_value = True
                continue
            # else:
                # print(" - Invalid {}".format(field))
        
        if not is_valid_value:
            invalid.append(value)
    
    return invalid

In [26]:
def scanning_error_rate(tickets, fields):
    invalid = []
    for ticket in tickets:
        invalid += check_ticket(ticket, fields)
    return sum(invalid)

## Example

In [15]:
with open("example.txt", "r") as file:
    example_notes = file.read()

In [16]:
example_fields, example_ticket, example_nearby = parse_notes(example_notes)

In [17]:
example_fields

{'class': [(1, 3), (5, 7)],
 'row': [(6, 11), (33, 44)],
 'seat': [(13, 40), (45, 50)]}

In [18]:
example_ticket

(7, 1, 14)

In [19]:
example_nearby

[(7, 3, 47), (40, 4, 50), (55, 2, 20), (38, 6, 12)]

In [22]:
assert check_ticket(example_nearby[0], example_fields) == []

In [23]:
assert check_ticket(example_nearby[1], example_fields) == [4]

In [24]:
assert check_ticket(example_nearby[2], example_fields) == [55]

In [25]:
assert check_ticket(example_nearby[3], example_fields) == [12]

In [29]:
assert scanning_error_rate(example_nearby, example_fields) == 71

## Part 1

In [31]:
with open("input.txt", "r") as file:
    input_fields, input_ticket, input_nearby = parse_notes(file.read().strip())

In [32]:
scanning_error_rate(input_nearby, input_fields)

32835

## Part 2

In [36]:
def only_valid(tickets, fields):
    valid_tickets = []
    for t in tickets:
        if len(check_ticket(t, fields)) == 0:
            valid_tickets.append(t)
    return valid_tickets

In [126]:
def predict_fields(tickets, fields):
    predictions = []
    for i in range(0, len(tickets[0])):
        possible = list(fields.keys())
        
        # eliminate possibilities from the fields list
        for ticket in tickets:
            for field in possible:
                if not check_field(ticket[i], fields[field]):
                    # the value cannot be one of the possible fields, so remove it
                    possible.remove(field)
        
        predictions.append(possible)
    
    definite_fields = [None] * len(tickets[0])
    predictions_updated = True
    
    while predictions_updated:
        predictions_updated = False
        for i, l in enumerate(predictions):
            if len(l) == 1 and len([p for p in predictions if p == l]):
                # if this position only has 1 prediction and no positions *just* have this prediction, then this position **must** be that field
                definite_fields[i] = l[0]
                
                # if this position is **definitly** a certain field, other positions cannot also be this field, so remove any predictions for this field
                # e.g. if this is definitely field A and the second position is predicted to be field A and B, then we can eliminate field A from the
                #      predictions for the second position
                new_predictions = []
                for p in predictions:
                    new_predictions.append([x for x in p if x != l[0]])
                predictions = new_predictions
                predictions_updated = True
    
    # make sure that all of the predictions are empty?
    for i, p in enumerate(predictions):
        if len(p) > 0:
            print("{} still has multiple predictions {}".format(i, p))
    
    return definite_fields

### Another example

In [37]:
with open("example2.txt", "r") as file:
    example2_fields, example2_ticket, example2_nearby = parse_notes(file.read().strip())

In [38]:
example2_valid = only_valid(example2_nearby, example2_fields)

In [127]:
x = predict_fields(example2_valid, example2_fields)

0 HAS to be ['row']
1 HAS to be ['class']
2 HAS to be ['seat']


In [128]:
x

['row', 'class', 'seat']

### And the real thing...

In [129]:
input_valid = only_valid(input_nearby, input_fields)

In [132]:
definite_fields = predict_fields(input_valid, input_fields)

12 HAS to be ['class']
9 HAS to be ['arrival location']
7 HAS to be ['arrival track']
11 HAS to be ['train']
10 HAS to be ['duration']
4 HAS to be ['seat']
3 HAS to be ['route']
16 HAS to be ['type']
6 HAS to be ['departure platform']
1 HAS to be ['departure time']
13 HAS to be ['departure track']
2 HAS to be ['departure station']
14 HAS to be ['departure date']
15 HAS to be ['departure location']
18 HAS to be ['arrival station']
17 HAS to be ['row']
5 HAS to be ['arrival platform']
0 HAS to be ['wagon']
8 HAS to be ['zone']
19 HAS to be ['price']


In [133]:
product = 1
for i, value in enumerate(input_ticket):
    print("{} -> {}".format(definite_fields[i], value))
    if definite_fields[i].startswith("departure "):
        product *= value

print("Answer: {}".format(product))

wagon -> 89
departure time -> 193
departure station -> 59
route -> 179
seat -> 191
arrival platform -> 173
departure platform -> 61
arrival track -> 73
zone -> 181
arrival location -> 67
duration -> 71
train -> 109
class -> 53
departure track -> 79
departure date -> 83
departure location -> 113
type -> 107
row -> 139
arrival station -> 131
price -> 137
Answer: 514662805187
